In [ ]:
from konlpy.tag import Okt
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# 한글 폰트 설정 (matplotlib에서 한글 표시를 위해)
plt.rcParams['font.family'] = 'Malgun Gothic'

def enhanced_text_preprocessing(text):
    """텍스트 전처리 강화"""
    if not text or pd.isna(text):
        return ""
    
    text = str(text)
    
    # 특수 문자 및 숫자 처리
    text = re.sub(r'[^\w\s가-힣a-zA-Z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def extract_nouns_from_text(text, okt):
    """텍스트에서 명사 추출"""
    if not text:
        return []
    
    # 명사 추출
    nouns = okt.nouns(text)
    
    # 단어 길이가 1 이하인 것 제거
    filtered_nouns = [noun for noun in nouns if len(noun) > 1]
    
    return filtered_nouns

def analyze_movie_overview():
    """영화 overview 데이터 분석"""
    
    # 1. 데이터 로드
    try:
        df = pd.read_csv('./tranning-data/movie_data.csv')
        print(f"데이터 로드 완료: {len(df)}개 행")
    except FileNotFoundError:
        print("movie_data.csv 파일을 찾을 수 없습니다.")
        return
    
    # 2. overview 컬럼 확인
    if 'overview' not in df.columns:
        print("overview 컬럼이 없습니다. 사용 가능한 컬럼:", df.columns.tolist())
        return
    
    # 3. 전처리
    print("텍스트 전처리 중...")
    df['processed_overview'] = df['overview'].apply(enhanced_text_preprocessing)
    
    # 4. 빈 텍스트 제거
    df_clean = df[df['processed_overview'].str.len() > 0].copy()
    print(f"전처리 후 유효한 데이터: {len(df_clean)}개")
    
    # 5. 명사 추출
    print("명사 추출 중...")
    okt = Okt()
    
    all_nouns = []
    for i, text in enumerate(df_clean['processed_overview']):
        if i % 100 == 0:
            print(f"진행률: {i}/{len(df_clean)}")
        
        nouns = extract_nouns_from_text(text, okt)
        all_nouns.extend(nouns)
    
    print(f"총 추출된 명사 개수: {len(all_nouns)}")
    
    # 6. 빈도 분석
    noun_counter = Counter(all_nouns)
    most_common_nouns = noun_counter.most_common(50)  # 상위 50개
    
    print("\n=== 상위 50개 명사 빈도 ===")
    for i, (noun, count) in enumerate(most_common_nouns, 1):
        print(f"{i:2d}. {noun}: {count}회")
    
    # # 7. 시각화
    # fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))
    
    # # 상위 20개 단어 막대 그래프
    # top_20_words = [word for word, count in most_common_nouns[:20]]
    # top_20_counts = [count for word, count in most_common_nouns[:20]]
    
    # ax1.barh(range(len(top_20_words)), top_20_counts)
    # ax1.set_yticks(range(len(top_20_words)))
    # ax1.set_yticklabels(top_20_words)
    # ax1.set_xlabel('빈도')
    # ax1.set_title('상위 20개 명사 빈도')
    # ax1.invert_yaxis()  # 높은 빈도가 위에 오도록
    
    # # 빈도별 분포 히스토그램
    # counts = list(noun_counter.values())
    # ax2.hist(counts, bins=50, edgecolor='black')
    # ax2.set_xlabel('단어 빈도')
    # ax2.set_ylabel('단어 개수')
    # ax2.set_title('단어 빈도 분포')
    # ax2.set_yscale('log')  # 로그 스케일로 표시
    
    # plt.tight_layout()
    # plt.show()
    
    # 8. 불용어 후보 추천
    print("\n=== 불용어 후보 추천 ===")
    print("다음 단어들은 불용어로 등록을 고려해보세요:")
    
    # 일반적인 불용어 후보들
    common_stopwords = ['영화', '작품', '이야기', '인물', '캐릭터', '시간', '세계', 
                       '모습', '상황', '과정', '경우', '때문', '문제', '방법', '결과',
                       '사람', '인간', '남자', '여자', '가족', '친구', '아이', '어린이']
    
    stopword_candidates = []
    for word, count in most_common_nouns[:30]:
        if word in common_stopwords or len(word) <= 2:
            stopword_candidates.append((word, count))
    
    for word, count in stopword_candidates:
        print(f"- {word} ({count}회)")
    
    # 9. 불용어 적용 예시
    print("\n=== 불용어 적용 후 결과 ===")
    
    # 불용어 리스트 (사용자가 수정 가능)
    stopwords = set([
                        '시간', '이런', '영화', '여자', '그것', '이야기', '방법', '이것', '저것', '결과', '문제', '어린이', '느낌', '경우',
                        '아이', '과정', '모습', '어떤', '무엇', '마음', '때문', '순간', '남자', '인간', '생각', '세계', '캐릭터', '하나',
                        '사람', '인물', '작품', '그런', '저런', '장면', '상황', '모든', '사실', '시작', '위해', '자신', '그녀', '서로',
                        '다른', '다시', '최고', '엄마', '아들', '아버지', '한편', '사이', '모두', '소녀', '점점', '아내', '간다', '마침내',
                        '바로', '이제', '이후', '처음', '마지막', '지금', '동안', '가운데', '잠시', '한번', '갑자기', '더욱', '거리', '달리', '아무',
                        '대한', '통해', '계속', '존재', '만난', '마주', '출신', '보고', '가지', '역시', '결정', '성격', '동네', '점차'
                        '대신', '우리', '혼자', '홀로', '자리', '도착', '이상', '매일', '그날', '과연', '누구', '가장', '이유', '도움', '만남', '기회',
                        '현실', '일상', '감정', '관계', '준비', '발생', '얼굴', '전화', '소리', '하루', '능력', '희망'
                        '등장', '진행', '제대로', '또한', '오늘', '아침', '최초', '마치', '게다가', '순식간', '다음', '현재', '이번', '지난', '직전', '본격',
                        '동료', '낯선', '주인', '애인', '배우', '달라', '자기', '부탁', '관심', '대해', 
                        '각자', '여기', '상대', '누군가', '얼마', '주변', '부모', '일행', '상태', '거듭', '멤버', '진짜', '불구', '선배', '후배',
                        '소식', '찾기', '외모', '행동', '선택', '절대', '로부터', '노력', '성공', '직감', 
                        '오직', '맞이', '매력', '상상', '타고', '자랑', '정신', '미소', '바람', '촬영', '감독', '스타',
                        '회사', '병원', '가게', '시골', '아파트', '집안', '무대', '방송', '추적', '공격', '목격', '해결', '접근', '포기', '시도',
                        '조사', '운영', '확인', '처리', '연락', '수행', '선화', '해리', '수현', '린다', '현우', '소희', '영수', '연희', '동수', '철민',
                        '은희', '지환', '상우', '송화', '피터', '진아', '영호', '동식', '현수', '태주', '상민', '희진', '유진', '도경', '민재', '스타크', '은주', '준석',
                        '목소리', '담당', '가슴', '이사', '머리', '반장', '대표', '전문', '눈치', '평생',
                        '사상', '최악', '실수', '방식', '목표', '뜻밖', '잡기', '직원', '사사건건', '정보', '머리'
                        '나선', '대가', '목적', '온갖', '전부', '어디', '직접', '당한', '보지', '가슴', '이사'
                        '확신', '지원', '요구', '몰래', '스스로', '녀석', '이용', '동시', '당황', '라서'
                        '가득', '데리', '차지', '서서히', '보이', '잡고', '로서', '오히려', '손님', '수업', '공개', '생애'
                        '보이지', '보기', '제일', '비롯', '이의', '당신', '영어', '라디오', '과장', '다섯'
                        '중인', '최후', '생애', '먼저', '전혀', '서서히', '설상가상', '보이', '보기', '보이지', '잡고'
                        '여정', '소원', '우연', '피해', '행복', '재회', '보호', '본능', '인기', '신분', '예측', '재능', '의지', '절망', '자유', '도전', '두려움', '생계', '강제', '계기',
                        '만큼', '거부', '욕망', '기쁨', '고백', '지목', '체포', '공부', '위장', '배달', '선언', '공개', '고민', '감지', '집착', '지배', '고립', '동원', '연기',
                    ])
    
                        
    # 불용어 제거 후 재분석
    filtered_nouns = [noun for noun in all_nouns if noun not in stopwords]
    filtered_counter = Counter(filtered_nouns)
    filtered_most_common = filtered_counter.most_common(400)
    
    print("불용어 제거 후 상위 50개 명사:")
    for i, (noun, count) in enumerate(filtered_most_common, 1):
        if count < 50:
            print(f"{i:2d}. {noun}: {count}회")
    
    return df_clean, noun_counter, filtered_counter, stopwords

# 실행
if __name__ == "__main__":
    df_clean, noun_counter, filtered_counter, stopwords = analyze_movie_overview()
    
    print("\n=== 분석 완료 ===")
    print(f"불용어 목록: {stopwords}")
    print("필요에 따라 불용어 목록을 수정하여 재분석할 수 있습니다.")

데이터 로드 완료: 1183개 행
텍스트 전처리 중...
전처리 후 유효한 데이터: 1180개
명사 추출 중...
진행률: 0/1180
진행률: 100/1180
진행률: 200/1180
진행률: 300/1180
진행률: 400/1180
진행률: 500/1180
진행률: 600/1180
진행률: 700/1180
진행률: 800/1180
진행률: 900/1180
진행률: 1000/1180
진행률: 1100/1180
총 추출된 명사 개수: 54599

=== 상위 50개 명사 빈도 ===
 1. 시작: 551회
 2. 위해: 521회
 3. 자신: 484회
 4. 사건: 456회
 5. 그녀: 431회
 6. 사람: 391회
 7. 사랑: 254회
 8. 친구: 236회
 9. 남자: 226회
10. 세상: 206회
11. 가족: 186회
12. 모든: 174회
13. 마음: 171회
14. 엄마: 164회
15. 서로: 163회
16. 아들: 157회
17. 여자: 153회
18. 하나: 148회
19. 다른: 148회
20. 시간: 140회
21. 살인: 138회
22. 아이: 136회
23. 형사: 134회
24. 다시: 132회
25. 아버지: 129회
26. 학교: 127회
27. 최고: 127회
28. 한편: 125회
29. 사이: 122회
30. 조직: 121회
31. 사실: 120회
32. 발견: 120회
33. 경찰: 115회
34. 모두: 114회
35. 소녀: 114회
36. 위기: 114회
37. 수사: 111회
38. 비밀: 102회
39. 사고: 102회
40. 생활: 101회
41. 순간: 101회
42. 마을: 100회
43. 아내: 98회
44. 생각: 98회
45. 죽음: 96회
46. 상황: 95회
47. 세계: 95회
48. 이자: 95회
49. 인생: 94회
50. 점점: 92회

=== 불용어 후보 추천 ===
다음 단어들은 불용어로 등록을 고려해보세요:
- 시작 (551회)
- 위해 (521회)
- 자신 (484회)
- 사건

In [ ]:
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np

# 한글 폰트 설정 (matplotlib에서 한글 표시를 위해)
plt.rcParams['font.family'] = 'Malgun Gothic'

def enhanced_text_preprocessing(text):
    """텍스트 전처리 강화"""
    if not text or pd.isna(text):
        return ""
    
    text = str(text)
    
    # 특수 문자 및 숫자 처리
    text = re.sub(r'[^\w\s가-힣a-zA-Z0-9]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

def extract_nouns_from_text(text, okt):
    """텍스트에서 명사 추출"""
    if not text:
        return []
    
    # 명사 추출
    nouns = okt.nouns(text)
    
    # 단어 길이가 1 이하인 것 제거
    filtered_nouns = [noun for noun in nouns if len(noun) > 1]
    
    return filtered_nouns

def analyze_movie_overview():
    """영화 overview 데이터 분석"""
    
    # 1. 데이터 로드
    try:
        df = pd.read_csv('./tranning-data/movie_data.csv')
        print(f"데이터 로드 완료: {len(df)}개 행")
    except FileNotFoundError:
        print("movie_data.csv 파일을 찾을 수 없습니다.")
        return
    
    # 2. overview 컬럼 확인
    if 'overview' not in df.columns:
        print("overview 컬럼이 없습니다. 사용 가능한 컬럼:", df.columns.tolist())
        return
    
    # 3. 전처리
    print("텍스트 전처리 중...")
    df['processed_overview'] = df['overview'].apply(enhanced_text_preprocessing)
    
    # 4. 빈 텍스트 제거
    df_clean = df[df['processed_overview'].str.len() > 0].copy()
    print(f"전처리 후 유효한 데이터: {len(df_clean)}개")
    
    # 5. 명사 추출
    print("명사 추출 중...")
    okt = Okt()
    
    all_nouns = []
    for i, text in enumerate(df_clean['processed_overview']):
        if i % 100 == 0:
            print(f"진행률: {i}/{len(df_clean)}")
        
        nouns = extract_nouns_from_text(text, okt)
        all_nouns.extend(nouns)
    
    print(f"총 추출된 명사 개수: {len(all_nouns)}")
    
    # 6. 빈도 분석
    noun_counter = Counter(all_nouns)
    most_common_nouns = noun_counter.most_common(500)  # 상위 50개
    
    print("\n=== 상위 50개 명사 빈도 ===")
    for i, (noun, count) in enumerate(most_common_nouns, 1):
        print(f"{i:2d}. {noun}: {count}회")

    
                        
    # 불용어 제거 후 재분석
    filtered_nouns = [noun for noun in all_nouns if noun not in stopwords]

    # TfidfVectorizer 객체 생성
    # min_df: 전체 문서 중 특정 단어가 나타나는 최소 문서 비율 또는 개수 (너무 희귀한 단어 제거)
    # max_df: 전체 문서 중 특정 단어가 나타나는 최대 문서 비율 또는 개수 (너무 흔한 단어 제거, 불용어 효과)
    # 이 값들을 조정하면서 최적의 결과를 찾아야 합니다.
    tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.8) # 예시 값

    # TF-IDF 행렬 학습 및 변환
    # processed_overview_for_tfidf: 각 줄거리에서 추출된 명사들을 공백으로 이은 문자열 리스트
    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_nouns)

    # IDF 값과 단어 매핑
    feature_names = tfidf_vectorizer.get_feature_names_out()
    idf_values = tfidf_vectorizer.idf_

    # IDF 값이 낮은 순서대로 정렬하여 불용어 후보 확인
    # IDF 값이 낮다는 것은 해당 단어가 여러 문서에 걸쳐 흔하게 등장한다는 의미 (불용어 후보)
    # IDF 값이 높다는 것은 해당 단어가 특정 문서에만 주로 등장하여 변별력이 높다는 의미 (키워드 후보)
    sorted_idf_indices = np.argsort(idf_values)
    low_idf_words = []

    print("\n=== IDF 값이 낮은 (흔한) 단어 목록 (불용어 후보) ===")
    # 상위 50개 정도의 IDF 낮은 단어들을 출력하여 수동 검토
    for i in range(min(500, len(feature_names))):
        idx = sorted_idf_indices[i]
        word = feature_names[idx]
        idf = idf_values[idx]
        print(f"{i+1:2d}. {word}: IDF={idf:.3f}")
        low_idf_words.append(word)

    print("\nTF-IDF 기반으로 추출된 불용어 후보 리스트 (낮은 IDF 값):")
    print(low_idf_words)
    
# 실행
if __name__ == "__main__":
    analyze_movie_overview()
    
    print("\n=== 분석 완료 ===")

데이터 로드 완료: 1183개 행
텍스트 전처리 중...
전처리 후 유효한 데이터: 1180개
명사 추출 중...
진행률: 0/1180
진행률: 100/1180
진행률: 200/1180
진행률: 300/1180
진행률: 400/1180
진행률: 500/1180
진행률: 600/1180
진행률: 700/1180
진행률: 800/1180
진행률: 900/1180
진행률: 1000/1180
진행률: 1100/1180
총 추출된 명사 개수: 54599

=== 상위 50개 명사 빈도 ===
 1. 시작: 551회
 2. 위해: 521회
 3. 자신: 484회
 4. 사건: 456회
 5. 그녀: 431회
 6. 사람: 391회
 7. 사랑: 254회
 8. 친구: 236회
 9. 남자: 226회
10. 세상: 206회
11. 가족: 186회
12. 모든: 174회
13. 마음: 171회
14. 엄마: 164회
15. 서로: 163회
16. 아들: 157회
17. 여자: 153회
18. 하나: 148회
19. 다른: 148회
20. 시간: 140회
21. 살인: 138회
22. 아이: 136회
23. 형사: 134회
24. 다시: 132회
25. 아버지: 129회
26. 학교: 127회
27. 최고: 127회
28. 한편: 125회
29. 사이: 122회
30. 조직: 121회
31. 사실: 120회
32. 발견: 120회
33. 경찰: 115회
34. 모두: 114회
35. 소녀: 114회
36. 위기: 114회
37. 수사: 111회
38. 비밀: 102회
39. 사고: 102회
40. 생활: 101회
41. 순간: 101회
42. 마을: 100회
43. 아내: 98회
44. 생각: 98회
45. 죽음: 96회
46. 상황: 95회
47. 세계: 95회
48. 이자: 95회
49. 인생: 94회
50. 점점: 92회
51. 때문: 92회
52. 대한: 91회
53. 바로: 86회
54. 마지막: 84회
55. 기억: 84회
56. 간다: 83회
57. 과거: 83회

In [14]:
general_stopwords = [
    "가게", "가운데", "가슴", "가지", "감정", "감독", "강제", "개가", "개월", "결과", "결정", "경우", "계기", "계속", "고객", "고민", "고백", "공개", "공격", "공부",
    "과장", "관계", "관심", "과정", "그것", "그녀", "그림", "그림자", "기분", "기쁨", "기운", "기회", "기업", "나는", "나선", "남자", "대가", "대신", "대해", "대표",
    "동료", "동네", "동수", "동원", "동행", "두려움", "드래곤", "등장", "라디오", "목소리", "마음", "만남", "매력", "멤버", "모두", "모습", "목표", "목적", "목격", "문제",
    "무대", "무엇", "미소", "반장", "발생", "방식", "방송", "배달", "배우", "배후", "백수", "부모", "부상", "부탁", "분위기", "분투", "비리", "병원", "보이", "보기", "보호",
    "보이지", "본능", "본부", "사건", "사상", "사신", "사실", "사회", "사이", "삶의", "상상", "상태", "상황", "선언", "선택", "성공", "성격", "세계", "소리", "소녀", "소식",
    "소원", "수업", "수행", "순간", "숨겨진", "스타", "시작", "시도", "신분", "신세", "신념", "신이", "신종", "실수", "실제", "실패", "심장", "아내", "아이", "아들", "아버지",
    "아파트", "악당", "애인", "액션", "어린이", "얼굴", "엄마", "에너지", "여자", "여정", "연기", "연락", "영주", "영웅", "예측", "오해", "온갖", "요구", "요청", "우리", "우연",
    "운영", "운명", "운동", "위기", "위로", "위장", "위치", "위험", "이야기", "이유", "이사", "이의", "인간", "인물", "인기", "인정", "입성", "자유", "자신", "자존심", "작가",
    "잡고", "잡기", "장면", "재능", "재회", "전화", "정보", "정신", "절망", "조작", "존재", "주변", "주인", "주인공", "주장", "준비", "중인", "중심", "지배", "지시", "지목",
    "지원", "직감", "직원", "진행", "집단", "집안", "집착", "차이", "차지", "추적", "최고", "최악", "치료", "친구", "촬영", "캐릭터", "코치", "타고", "뜻밖", "파괴", "파트너",
    "평생", "평화", "포기", "피해", "피해자", "하나", "하룻밤", "하루", "해결", "행동", "행복", "행방", "현실", "현수", "현우", "희망", "회사", "훈련", "확인", "확신", "환경",
    "혐의", "호기심", "혼란", "혼자"
]


stopwords_set = set(general_stopwords)

# 파일을 쓰기 모드(w)로 엽니다. 'utf-8' 인코딩을 사용합니다.
with open('general_stopwords.txt', 'w', encoding='utf-8') as f: # 파일명을 'general_stopwords.txt'로 통일
    for word in sorted(list(stopwords_set)):
        f.write(word + '\n')

print("불용어 목록이 'general_stopwords.txt' 파일로 성공적으로 저장되었습니다.")

불용어 목록이 'general_stopwords.txt' 파일로 성공적으로 저장되었습니다.


In [15]:
korean_adverb_stopwords = [
    "가득", "갈수록", "강력", "거듭", "거기", "결국", "결코", "게다가",
    "과연", "그때", "난생", "누구든", "누군가", "누군지", "누구", "단지",
    "달리", "당일", "더욱", "도저히", "도중", "동시", "듯이", "마냥",
    "마침내", "막상", "만큼", "몰래", "무슨", "무턱대고", "물론", "미처",
    "바닥", "바로", "번의", "본격", "불구", "비롯", "사사건건", "상대",
    "서서히", "설마", "설상가상", "순식간", "아무", "아무도", "아홉", "아침",
    "안고", "앞서", "어디", "어딘가", "얼마", "얼떨결", "역시", "역설적",
    "연쇄", "오히려", "오직", "일단", "일쑤", "일종의", "이미", "이사",
    "이후", "자칫", "자꾸", "전부", "전혀", "절대", "점점", "점차",
    "정도의", "제대로", "직전", "직접", "진짜", "차라리", "차마", "최초",
    "최후", "항상", "한번", "한창", "혹시", "홀로", "갑자기", "계속",
    "비로소", "조금씩", "단숨에", "무심코", "그대로", "다시", "먼저", "함께",
    "굳이", "단단히", "종종", "확실히", "살며시", "불쑥", "슬쩍", "기어이",
    "몹시", "참으로", "그리", "이리", "저리", "결정적으로", "필연적으로", "늘",
    "아주", "매우", "정말", "어쩐지", "어떻게든", "대충", "도대체", "가끔",
    "때때로", "계속해서", "간신히", "마주", "은근히", "불현듯", "재빨리", "흔히",
    "급히", "가만히", "조용히", "순간적으로", "느닷없이", "덜컥", "툭", "확",
    "딱", "살짝", "쩍", "휙", "펑", "콩", "꽝", "툭툭",
    "속속", "반드시", "현재", "아직", "뒤늦게", "훨씬", "그나마", "이내",
    "얼른", "곧", "빨리", "늦게", "슬슬", "어느새", "차츰", "부쩍",
    "무척", "아예", "잇따라", "사이좋게", "제멋대로", "맘대로", "마음껏", "함부로",
    "조심스럽게", "조심히", "정중히", "적극적으로", "신속히", "즉시", "한참", "끈질기게",
    "무리하게", "대대적으로", "과감히", "과감하게", "천천히", "서둘러", "바삐", "무작정",
    "필사적으로", "요란하게", "불쌍하게", "간절히", "간절하게", "단호히", "엄숙하게", "엄중하게",
    "기쁘게", "즐겁게", "행복하게", "슬프게", "괴롭게", "고통스럽게", "미련없이", "깨끗이",
    "흠뻑", "잔뜩", "제법", "조금", "약간", "충분히", "흐릿하게", "뚜렷하게",
    "명확하게", "확연히", "대충대충", "빙글빙글", "우르르", "줄줄", "방긋", "깜짝",
    "살포시", "바짝", "꼼꼼히", "차근차근", "뚝딱", "똑똑히", "신중히", "순순히",
    "적당히", "지그시", "하염없이", "그럭저럭", "어렴풋이", "맑게", "단숨", "단둘",
    "어디선가", "일찍이", "예전에", "어느덧"
]

korean_adverb_stopwords_set = set(korean_adverb_stopwords)

# 파일을 쓰기 모드(w)로 엽니다. 'utf-8' 인코딩을 사용합니다.
with open('korean_adverb_stopwords.txt', 'w', encoding='utf-8') as f: # 파일명을 'korean_adverb_stopwords.txt'로 통일
    for word in sorted(list(korean_adverb_stopwords)):
        f.write(word + '\n')

print("불용어 목록이 'korean_adverb_stopwords.txt' 파일로 성공적으로 저장되었습니다.")

불용어 목록이 'korean_adverb_stopwords.txt' 파일로 성공적으로 저장되었습니다.


In [16]:
common_name_stopwords = [
    "수진", "민수", "지훈", "영희", "철수", "지민", "은우", "지수", "서연", "현우",
    "나영", "수현", "도현", "서준", "유진", "정우", "하늘", "하람", "연우", "보라",
    "지은", "진우", "유나", "가영", "혜진", "하은", "채원", "예린", "주원", "세영",
    "유리", "수빈", "시우", "지아", "다은", "세준", "민재", "민지", "태현", "하윤",
    "서윤", "윤아", "성민", "승현", "아린", "은비", "지안", "도윤", "소율", "민혁",
    "예준", "현수", "유정", "시현", "예원", "민성", "주아", "수아", "다연", "승민",
    "유찬", "시은", "하린", "윤호", "유빈", "서하", "가은", "진아", "성훈", "민호",
    "예진", "태양", "은서", "지원", "연서", "유하", "시윤", "예빈", "주하", "정민",
    "유나", "예나", "주희", "채린", "서우", "지호", "준호", "예슬", "태희", "은채",
    "윤서", "지연", "서영", "지현", "은정", "나은", "유림", "정현", "소현", "소연",
    "준영", "세아", "다희", "유희", "혜원", "윤정", "은영", "미나", "보영", "지윤",
    "현정", "예영", "지우", "미영", "수영", "재영", "정아", "희진", "영수", "상우",
    "지혜", "은주", "승아", "정호", "동혁", "지성", "정은", "정수", "강호", "세진",
    "상민", "태주", "동수", "동식", "상희", "도경", "형민", "지석", "명훈", "석현",
    "기현", "재민", "종현", "영호", "준석", "준수", "건우", "도훈", "재석", "호준",
    "지호", "시훈", "정환", "태현", "민경", "재희", "상훈", "정우", "혁진", "동현",
    "소민", "지환", "현민", "재욱", "재호", "은지", "시영", "채민", "유선", "명진",
    "서진", "하영", "가희", "윤지", "도연", "재경", "영주", "미정", "정재", "하진",
    "진혁", "정빈", "세영", "명수", "영석", "준형", "영기", "은하", "기태", "우진",
    "지성", "정태", "한결", "태연", "성호", "시진", "연우", "현경", "현서", "민희",
    "보현", "세희", "예림", "소라", "하정", "유라", "현진", "하윤", "서율", "유림",
    "구창", "경민", "건우", "린다", "민아", "민재", "미영", "미진", "상민", "상우",
    "선민", "선영", "선화", "소연", "소희", "송화", "수진", "수인", "수정", "수현",
    "영수", "영주", "영호", "용준", "우진", "윤희", "은수", "은주", "은희", "재영",
    "준석", "준영", "지환", "지훈", "진아", "진성", "철민", "태수", "태주", "피터",
    "한수", "해주", "현수", "현우", "희수", "희진", "유진", "도경", "스타크"
]

common_name_stopwords_set = set(common_name_stopwords)

# 파일을 쓰기 모드(w)로 엽니다. 'utf-8' 인코딩을 사용합니다.
with open('common_name_stopwords.txt', 'w', encoding='utf-8') as f: # 파일명을 'common_name_stopwords_set.txt'로 통일
    for word in sorted(list(common_name_stopwords_set)):
        f.write(word + '\n')

print("불용어 목록이 'common_name_stopwords.txt' 파일로 성공적으로 저장되었습니다.")

불용어 목록이 'common_name_stopwords.txt' 파일로 성공적으로 저장되었습니다.


In [12]:
loaded_stopwords = set()
# 파일을 읽기 모드(r)로 엽니다. 'utf-8' 인코딩을 사용합니다.
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    for line in f:
        # 각 줄의 개행 문자(줄바꿈)를 제거하고 불용어 집합에 추가합니다.
        loaded_stopwords.add(line.strip())

print(f"불용어 목록이 'stopwords.txt' 파일에서 성공적으로 불러와졌습니다. 총 {len(loaded_stopwords)}개 단어.")
# 불러온 불용어 중 일부를 출력하여 확인
print(list(loaded_stopwords)[:10])

불용어 목록이 'stopwords.txt' 파일에서 성공적으로 불러와졌습니다. 총 400개 단어.
['직감', '잡고', '사실', '무엇', '유진', '역설적', '혹시', '왠지', '진짜', '맞이']
